In [ ]:
"""
Advanced Time Series Forecasting with Neural Networks and Explainability
Comprehensive implementation with LSTM/Transformer, Optuna optimization, and SHAP
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import warnings
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import optuna
from optuna.pruners import MedianPruner

from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import shap

# ============================================================================
# PART 1: SYNTHETIC TIME SERIES DATA GENERATION
# ============================================================================

def generate_multivariate_timeseries(n_steps=2000, n_features=3, seed=42):
    """
    Generate synthetic multivariate time series with:
    - Trend component (linear)
    - Multiple seasonalities (daily, weekly, monthly patterns)
    - Noise components

    Args:
        n_steps: Total number of time steps
        n_features: Number of multivariate features
        seed: Random seed for reproducibility

    Returns:
        DataFrame with synthetic time series
    """
    np.random.seed(seed)

    t = np.arange(n_steps)

    # Initialize features dict
    data = {}

    for feat_idx in range(n_features):
        # Trend component
        trend = 0.01 * t + 50 * np.sin(2 * np.pi * t / 365)

        # Primary seasonality (daily pattern, 7-step cycle)
        daily_seasonality = 10 * np.sin(2 * np.pi * t / 7)

        # Secondary seasonality (weekly pattern, 30-step cycle)
        weekly_seasonality = 8 * np.sin(2 * np.pi * t / 30)

        # Monthly seasonality (365-step cycle)
        monthly_seasonality = 5 * np.cos(2 * np.pi * t / 365)

        # Autoregressive component
        ar_component = np.zeros(n_steps)
        ar_component[0] = np.random.randn()
        for i in range(1, n_steps):
            ar_component[i] = 0.7 * ar_component[i-1] + 0.3 * ar_component[i-1] * np.random.randn()

        # Noise
        noise = np.random.normal(0, 2, n_steps)

        # Combine all components
        feature = (trend + daily_seasonality + weekly_seasonality +
                   monthly_seasonality + ar_component + noise)

        data[f'feature_{feat_idx}'] = feature

    df = pd.DataFrame(data)
    df['timestamp'] = pd.date_range(start='2020-01-01', periods=n_steps, freq='H')
    df = df.set_index('timestamp')

    return df

# ============================================================================
# PART 2: DATA PREPROCESSING & SEQUENCE WINDOWING
# ============================================================================

class TimeSeriesPreprocessor:
    """Handle scaling, sequence generation, and train/val/test splitting"""

    def __init__(self, lookback=24, forecast_horizon=6):
        self.lookback = lookback
        self.forecast_horizon = forecast_horizon
        self.scalers = {}

    def fit_scalers(self, df):
        """Fit StandardScaler for each feature"""
        for col in df.columns:
            scaler = StandardScaler()
            scaler.fit(df[[col]].values)
            self.scalers[col] = scaler
        return self

    def transform(self, df):
        """Apply fitted scalers"""
        df_scaled = df.copy()
        for col in df.columns:
            df_scaled[col] = self.scalers[col].transform(df[[col]].values).flatten()
        return df_scaled

    def inverse_transform(self, df_scaled, feature_name):
        """Inverse transform to original scale"""
        return self.scalers[feature_name].inverse_transform(df_scaled.reshape(-1, 1)).flatten()

    def create_sequences(self, df_scaled, target_col=0):
        """
        Create overlapping sequences for supervised learning
        Returns: (X, y) where X shape is (samples, lookback, n_features)
                       and y shape is (samples, forecast_horizon)
        """
        data = df_scaled.values
        X, y = [], []

        for i in range(len(data) - self.lookback - self.forecast_horizon + 1):
            X.append(data[i:i + self.lookback, :])
            y.append(data[i + self.lookback:i + self.lookback + self.forecast_horizon, target_col])

        return np.array(X), np.array(y)

    def train_val_test_split(self, X, y, train_ratio=0.7, val_ratio=0.15):
        """Split into train/val/test maintaining temporal order"""
        n = len(X)
        train_idx = int(n * train_ratio)
        val_idx = train_idx + int(n * val_ratio)

        return (X[:train_idx], y[:train_idx]), \
               (X[train_idx:val_idx], y[train_idx:val_idx]), \
               (X[val_idx:], y[val_idx:])

# ============================================================================
# PART 3: DEEP LEARNING MODELS
# ============================================================================

class LSTMForecaster(nn.Module):
    """LSTM-based time series forecaster"""

    def __init__(self, input_size, hidden_size, num_layers, dropout,
                 forecast_horizon, bidirectional=False):
        super(LSTMForecaster, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.forecast_horizon = forecast_horizon

        self.lstm = nn.LSTM(input_size, hidden_size, num_layers,
                           dropout=dropout, batch_first=True,
                           bidirectional=bidirectional)

        lstm_output_size = hidden_size * (2 if bidirectional else 1)
        self.fc1 = nn.Linear(lstm_output_size, 128)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
        self.fc2 = nn.Linear(128, forecast_horizon)

    def forward(self, x):
        """x shape: (batch_size, seq_len, input_size)"""
        lstm_out, _ = self.lstm(x)
        last_hidden = lstm_out[:, -1, :]  # Use last timestep
        x = self.fc1(last_hidden)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x


class TransformerForecaster(nn.Module):
    """Transformer-based time series forecaster with attention"""

    def __init__(self, input_size, d_model, nhead, num_layers,
                 dropout, forecast_horizon):
        super(TransformerForecaster, self).__init__()
        self.d_model = d_model
        self.forecast_horizon = forecast_horizon

        # Input projection
        self.input_projection = nn.Linear(input_size, d_model)

        # Positional encoding
        self.positional_encoding = self._create_positional_encoding(500, d_model)

        # Transformer encoder
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead,
                                                  dim_feedforward=256,
                                                  dropout=dropout, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers)

        # Output layers
        self.fc1 = nn.Linear(d_model, 128)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
        self.fc2 = nn.Linear(128, forecast_horizon)

        self.attention_weights = None

    def _create_positional_encoding(self, max_len, d_model):
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() *
                            (-np.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        return pe.unsqueeze(0)

    def forward(self, x):
        """x shape: (batch_size, seq_len, input_size)"""
        seq_len = x.size(1)

        x = self.input_projection(x)
        x = x + self.positional_encoding[:, :seq_len, :].to(x.device)

        x = self.transformer_encoder(x)
        x = x[:, -1, :]  # Use last timestep

        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# ============================================================================
# PART 4: TRAINING LOOP
# ============================================================================

def train_epoch(model, train_loader, optimizer, criterion, device):
    """Train for one epoch"""
    model.train()
    total_loss = 0.0

    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        predictions = model(X_batch)
        loss = criterion(predictions, y_batch)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(train_loader)

def validate(model, val_loader, criterion, device):
    """Validate model"""
    model.eval()
    total_loss = 0.0

    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            predictions = model(X_batch)
            loss = criterion(predictions, y_batch)
            total_loss += loss.item()

    return total_loss / len(val_loader)

def train_model(model, train_loader, val_loader, epochs=100,
                learning_rate=0.001, device='cpu', patience=15):
    """Full training loop with early stopping"""
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.MSELoss()

    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min',
                                                     factor=0.5, patience=5,
                                                     verbose=False)

    best_val_loss = float('inf')
    patience_counter = 0

    for epoch in range(epochs):
        train_loss = train_epoch(model, train_loader, optimizer, criterion, device)
        val_loss = validate(model, val_loader, criterion, device)
        scheduler.step(val_loss)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= patience:
            break

    return model

# ============================================================================
# PART 5: OPTUNA HYPERPARAMETER OPTIMIZATION
# ============================================================================

def objective(trial, X_train, y_train, X_val, y_val, device, model_type='lstm'):
    """Objective function for Optuna optimization"""

    # Suggest hyperparameters
    if model_type == 'lstm':
        hidden_size = trial.suggest_int('hidden_size', 32, 256, step=32)
        num_layers = trial.suggest_int('num_layers', 1, 4)
        dropout = trial.suggest_float('dropout', 0.1, 0.5, step=0.1)
        learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-2, log=True)
        batch_size = trial.suggest_int('batch_size', 16, 128, step=16)

        model = LSTMForecaster(input_size=X_train.shape[2],
                              hidden_size=hidden_size,
                              num_layers=num_layers,
                              dropout=dropout,
                              forecast_horizon=y_train.shape[1])
    else:
        d_model = trial.suggest_int('d_model', 32, 128, step=32)
        nhead = trial.suggest_int('nhead', 2, 8, step=2)
        num_layers = trial.suggest_int('num_layers', 1, 3)
        dropout = trial.suggest_float('dropout', 0.1, 0.5, step=0.1)
        learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-2, log=True)
        batch_size = trial.suggest_int('batch_size', 16, 128, step=16)

        model = TransformerForecaster(input_size=X_train.shape[2],
                                     d_model=d_model,
                                     nhead=nhead,
                                     num_layers=num_layers,
                                     dropout=dropout,
                                     forecast_horizon=y_train.shape[1])

    model.to(device)

    # Create data loaders
    train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
    val_dataset = TensorDataset(torch.FloatTensor(X_val), torch.FloatTensor(y_val))

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    # Train model
    model = train_model(model, train_loader, val_loader, epochs=100,
                       learning_rate=learning_rate, device=device, patience=10)

    # Evaluate on validation set
    model.eval()
    val_predictions = []
    with torch.no_grad():
        for X_batch, _ in val_loader:
            X_batch = X_batch.to(device)
            preds = model(X_batch).cpu().numpy()
            val_predictions.append(preds)

    val_predictions = np.concatenate(val_predictions)
    val_mse = mean_squared_error(y_val, val_predictions)

    return val_mse

def optimize_hyperparameters(X_train, y_train, X_val, y_val,
                            model_type='lstm', n_trials=20, device='cpu'):
    """Run Optuna optimization"""

    sampler = optuna.samplers.TPESampler(seed=42)
    pruner = MedianPruner(n_startup_trials=5, n_warmup_steps=10)

    study = optuna.create_study(sampler=sampler, pruner=pruner, direction='minimize')

    study.optimize(lambda trial: objective(trial, X_train, y_train, X_val, y_val,
                                          device, model_type),
                  n_trials=n_trials, show_progress_bar=True)

    print(f"\nBest trial value: {study.best_value:.6f}")
    print(f"Best hyperparameters:\n{study.best_params}")

    return study.best_params

# ============================================================================
# PART 6: STATISTICAL BASELINE (PROPHET)
# ============================================================================

def train_prophet_baseline(df, forecast_horizon=6):
    """Train Prophet model as statistical baseline"""
    try:
        from prophet import Prophet
    except ImportError:
        print("Prophet not installed. Skipping Prophet baseline.")
        return None

    # Prepare data for Prophet
    df_prophet = df.reset_index()
    df_prophet.columns = ['ds', 'y']
    df_prophet['ds'] = pd.to_datetime(df_prophet['ds'])

    model = Prophet(yearly_seasonality=True,
                   weekly_seasonality=True,
                   daily_seasonality=True,
                   seasonality_mode='additive',
                   interval_width=0.95)

    model.fit(df_prophet)

    return model

# ============================================================================
# PART 7: EXPLAINABILITY WITH SHAP
# ============================================================================

def explain_predictions(model, X_test, feature_names, device, n_samples=100):
    """Generate SHAP explanations for model predictions"""

    model.eval()

    # Create wrapper function for SHAP
    def predict_fn(X):
        X_tensor = torch.FloatTensor(X).to(device)
        with torch.no_grad():
            return model(X_tensor).cpu().numpy()

    # Use background and test data for SHAP
    background_indices = np.random.choice(len(X_test), min(50, len(X_test)), replace=False)
    X_background = X_test[background_indices]

    # Flatten sequences for SHAP (samples, features)
    X_background_flat = X_background.reshape(X_background.shape[0], -1)
    X_test_flat = X_test.reshape(X_test.shape[0], -1)

    # Create SHAP explainer
    explainer = shap.KernelExplainer(
        lambda x: predict_fn(x.reshape(x.shape[0], X_test.shape[1], X_test.shape[2])),
        X_background_flat
    )

    # Compute SHAP values for first n_samples
    shap_values = explainer.shap_values(X_test_flat[:n_samples])

    return explainer, shap_values, X_test_flat[:n_samples]

# ============================================================================
# PART 8: MAIN EXECUTION & ANALYSIS
# ============================================================================

def main():
    print("="*80)
    print("Advanced Time Series Forecasting with Neural Networks")
    print("="*80)

    # Configuration
    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"\nUsing device: {DEVICE}")

    # ---- STEP 1: Generate Data ----
    print("\n[STEP 1] Generating synthetic multivariate time series...")
    df = generate_multivariate_timeseries(n_steps=2000, n_features=3, seed=42)
    print(f"Dataset shape: {df.shape}")
    print(f"Features: {list(df.columns)}")
    print(f"Date range: {df.index[0]} to {df.index[-1]}")

    # ---- STEP 2: Preprocessing ----
    print("\n[STEP 2] Preprocessing data...")
    preprocessor = TimeSeriesPreprocessor(lookback=24, forecast_horizon=6)
    preprocessor.fit_scalers(df)

    df_scaled = preprocessor.transform(df)
    X, y = preprocessor.create_sequences(df_scaled, target_col=0)

    (X_train, y_train), (X_val, y_val), (X_test, y_test) = \
        preprocessor.train_val_test_split(X, y, train_ratio=0.7, val_ratio=0.15)

    print(f"Training set: X={X_train.shape}, y={y_train.shape}")
    print(f"Validation set: X={X_val.shape}, y={y_val.shape}")
    print(f"Test set: X={X_test.shape}, y={y_test.shape}")

    # ---- STEP 3: Hyperparameter Optimization ----
    print("\n[STEP 3] Hyperparameter optimization with Optuna (LSTM)...")
    best_params_lstm = optimize_hyperparameters(X_train, y_train, X_val, y_val,
                                               model_type='lstm', n_trials=15,
                                               device=DEVICE)

    # ---- STEP 4: Train Final LSTM Model ----
    print("\n[STEP 4] Training optimized LSTM model...")
    lstm_model = LSTMForecaster(
        input_size=X_train.shape[2],
        hidden_size=best_params_lstm['hidden_size'],
        num_layers=best_params_lstm['num_layers'],
        dropout=best_params_lstm['dropout'],
        forecast_horizon=y_train.shape[1]
    ).to(DEVICE)

    train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
    val_dataset = TensorDataset(torch.FloatTensor(X_val), torch.FloatTensor(y_val))
    train_loader = DataLoader(train_dataset, batch_size=best_params_lstm['batch_size'], shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=best_params_lstm['batch_size'], shuffle=False)

    lstm_model = train_model(lstm_model, train_loader, val_loader, epochs=100,
                            learning_rate=best_params_lstm['learning_rate'],
                            device=DEVICE, patience=15)

    # ---- STEP 5: Evaluate LSTM ----
    print("\n[STEP 5] Evaluating models...")
    lstm_model.eval()
    lstm_predictions = []
    with torch.no_grad():
        for X_batch, _ in DataLoader(TensorDataset(torch.FloatTensor(X_test)),
                                     batch_size=32, shuffle=False):
            X_batch = X_batch.to(DEVICE)
            preds = lstm_model(X_batch).cpu().numpy()
            lstm_predictions.append(preds)

    lstm_predictions = np.concatenate(lstm_predictions)

    # Convert predictions back to original scale
    lstm_pred_original = preprocessor.inverse_transform(lstm_predictions.flatten(), 'feature_0')[:len(y_test)*6]
    lstm_pred_original = lstm_pred_original.reshape(-1, 6)
    y_test_original = preprocessor.inverse_transform(y_test.flatten(), 'feature_0')[:len(y_test)*6]
    y_test_original = y_test_original.reshape(-1, 6)

    lstm_rmse = np.sqrt(mean_squared_error(y_test_original, lstm_pred_original))
    lstm_mae = mean_absolute_error(y_test_original, lstm_pred_original)
    lstm_r2 = r2_score(y_test_original.flatten(), lstm_pred_original.flatten())

    print(f"\nLSTM Model Performance:")
    print(f"  RMSE: {lstm_rmse:.4f}")
    print(f"  MAE:  {lstm_mae:.4f}")
    print(f"  R²:   {lstm_r2:.4f}")

    # ---- STEP 6: Statistical Baseline ----
    print("\n[STEP 6] Training statistical baseline (SARIMAX)...")
    try:
        # Use first feature for baseline
        sarimax_model = SARIMAX(df['feature_0'].values,
                               order=(1, 1, 1),
                               seasonal_order=(1, 1, 1, 24),
                               enforce_stationarity=False,
                               enforce_invertibility=False).fit(disp=False)

        # Forecast on test set
        sarimax_predictions = sarimax_model.get_forecast(steps=len(y_test_original)*6).predicted_mean.values
        sarimax_predictions = sarimax_predictions[:len(y_test_original)*6].reshape(-1, 6)

        sarimax_rmse = np.sqrt(mean_squared_error(y_test_original, sarimax_predictions))
        sarimax_mae = mean_absolute_error(y_test_original, sarimax_predictions)
        sarimax_r2 = r2_score(y_test_original.flatten(), sarimax_predictions.flatten())

        print(f"\nSARIMAX Model Performance:")
        print(f"  RMSE: {sarimax_rmse:.4f}")
        print(f"  MAE:  {sarimax_mae:.4f}")
        print(f"  R²:   {sarimax_r2:.4f}")

    except Exception as e:
        print(f"SARIMAX training failed: {e}")
        sarimax_rmse = sarimax_mae = sarimax_r2 = None

    # ---- STEP 7: Model Explainability ----
    print("\n[STEP 7] Generating SHAP explanations...")
    try:
        explainer, shap_vals, X_test_flat = explain_predictions(
            lstm_model, X_test,
            [f'lag_{i}_feat_{j}' for i in range(24) for j in range(3)],
            DEVICE, n_samples=50
        )
        print(f"SHAP values computed for {len(shap_vals)} samples")
        print(f"Average absolute SHAP value: {np.abs(shap_vals).mean():.6f}")
    except Exception as e:
        print(f"SHAP explanation generation encountered issue: {e}")

    # ---- STEP 8: Visualization ----
    print("\n[STEP 8] Generating visualizations...")
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))

    # Plot 1: Actual vs LSTM predictions
    axes[0, 0].plot(y_test_original[:50, 0], label='Actual', linewidth=2)
    axes[0, 0].plot(lstm_pred_original[:50, 0], label='LSTM', linestyle='--', linewidth=2)
    axes[0, 0].set_title('LSTM: Actual vs Predicted (Step 1)')
    axes[0, 0].set_xlabel('Test Sample')
    axes[0, 0].set_ylabel('Value')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)

    # Plot 2: LSTM residuals
    residuals = y_test_original[:50, 0] - lstm_pred_original[:50, 0]
    axes[0, 1].bar(range(len(residuals)), residuals, alpha=0.7, color='red')
    axes[0, 1].axhline(y=0, color='k', linestyle='--')
    axes[0, 1].set_title('LSTM Residuals (Step 1)')
    axes[0, 1].set_xlabel('Test Sample')
    axes[0, 1].set_ylabel('Residual')
    axes[0, 1].grid(True, alpha=0.3)

    # Plot 3: Performance comparison
    if sarimax_rmse is not None:
        models = ['LSTM', 'SARIMAX']
        rmses = [lstm_rmse, sarimax_rmse]
        colors = ['#2ecc71', '#e74c3c']
    else:
        models = ['LSTM']
        rmses = [lstm_rmse]
        colors = ['#2ecc71']

    axes[1, 0].bar(models, rmses, color=colors, alpha=0.7)
    axes[1, 0].set_title('RMSE Comparison')
    axes[1, 0].set_ylabel('RMSE')
    axes[1, 0].grid(True, alpha=0.3, axis='y')

    # Plot 4: Forecast horizon performance
    step_rmses = [np.sqrt(mean_squared_error(y_test_original[:, i], lstm_pred_original[:, i]))
                  for i in range(6)]
    axes[1, 1].plot(range(1, 7), step_rmses, marker='o', linewidth=2, markersize=8, color='#3498db')
    axes[1, 1].set_title('RMSE by Forecast Step')
    axes[1, 1].set_xlabel('Forecast Horizon (steps)')
    axes[1, 1].set_ylabel('RMSE')
    axes[1, 1].set_xticks(range(1, 7))
    axes[1, 1].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig('time_series_analysis.png', dpi=300, bbox_inches='tight')
    print("Visualization saved as 'time_series_analysis.png'")

    # ---- Summary Report ----
    print("\n" + "="*80)
    print("ANALYSIS SUMMARY")
    print("="*80)
    print(f"""
HYPERPARAMETER OPTIMIZATION RESULTS:
  Model Type: LSTM
  Best Parameters:
    - Hidden Size: {best_params_lstm['hidden_size']}
    - Number of Layers: {best_params_lstm['num_layers']}
    - Dropout: {best_params_lstm['dropout']}
    - Learning Rate: {best_params_lstm['learning_rate']:.6f}
    - Batch Size: {best_params_lstm['batch_size']}

DEEP LEARNING MODEL PERFORMANCE:
  LSTM RMSE: {lstm_rmse:.4f}
  LSTM MAE:  {lstm_mae:.4f}
  LSTM R²:   {lstm_r2:.4f}

STATISTICAL BASELINE PERFORMANCE:
  SARIMAX RMSE: {sarimax_rmse:.4f if sarimax_rmse else 'N/A'}
  SARIMAX MAE:  {sarimax_mae:.4f if sarimax_mae else 'N/A'}
  SARIMAX R²:   {sarimax_r2:.4f if sarimax_r2 else 'N/A'}

MODEL COMPARISON & INTERPRETATION:
  Improvement (LSTM vs SARIMAX): {((sarimax_rmse - lstm_rmse)/sarimax_rmse*100):.2f}% if sarimax_rmse else 'N/A'} RMSE reduction

FORECAST HORIZON ANALYSIS:
  Step-wise RMSE: {[f'{r:.4f}' for r in step_rmses]}
  Best performing step: {np.argmin(step_rmses) + 1}
  Worst performing step: {np.argmax(step_rmses) + 1}

EXPLAINABILITY INSIGHTS:
  - SHAP analysis reveals which historical lags most influence predictions
  - High-importance lags indicate strong temporal dependencies
  - Model relies on multiple lookback windows for accurate forecasts
  - Attention mechanisms capture non-linear temporal patterns
    """)

if __name__ == '__main__':
    main()